In [6]:
import os
import os.path as path

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

MODEL_NAME = '1608_mnist'

''' 
loads mnist data 
reshapes train and test data
converts their types to float
divides them by 255 to scale data to fit between 0 and 1
converts to one hot encoding
returns data and labels for both training and test
'''
def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    return x_train, y_train, x_test, y_test

'''
Creates the structure of our model
'''
def build_model():
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=3, strides=1, \
            padding='same', activation='relu', \
            input_shape=[28, 28, 1]))
    # 28*28*64
    
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 14*14*64

    model.add(Conv2D(filters=128, kernel_size=3, strides=1, \
            padding='same', activation='relu'))
    # 14*14*128
    
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 7*7*128

    model.add(Conv2D(filters=256, kernel_size=3, strides=1, \
            padding='same', activation='relu'))
    # 7*7*256
    
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 4*4*256

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

'''
trains our model with given data

categorical-cross-entropy
Adadelta
20 epochs
128 batch size
'''
def train(model, x_train, y_train, x_test, y_test):
    model.compile(loss=keras.losses.categorical_crossentropy, \
                  optimizer=keras.optimizers.Adadelta(), \
                  metrics=['accuracy'])

    model.fit(x_train, y_train, \
              batch_size=128, \
              epochs=20, \
              verbose=1, \
              validation_data=(x_test, y_test))

'''
Freezes model variables, saves the model as protobuf file
So that it is usable in Android devices.
'''
def export_model(saver, model, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")


if not path.exists('out'):
    os.mkdir('out')

x_train, y_train, x_test, y_test = load_data()

model = build_model()

train(model, x_train, y_train, x_test, y_test)

'''
While saving our model, we need to specify input node and output node. Therefore before
calling export_model method, we need to summarize our model to see the names of these nodes
'''
print(model.summary())
    

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 11s 183us/step - loss: 0.2243 - acc: 0.9283 - val_loss: 0.0400 - val_acc: 0.9869
Epoch 2/20
60000/60000 [==============================] - 10s 163us/step - loss: 0.0417 - acc: 0.9867 - val_loss: 0.0291 - val_acc: 0.9902
Epoch 3/20
60000/60000 [==============================] - 10s 162us/step - loss: 0.0275 - acc: 0.9914 - val_loss: 0.0262 - val_acc: 0.9909
Epoch 4/20
60000/60000 [==============================] - 10s 162us/step - loss: 0.0192 - acc: 0.9941 - val_loss: 0.0240 - val_acc: 0.9920
Epoch 5/20
60000/60000 [==============================] - 10s 163us/step - loss: 0.0132 - acc: 0.9959 - val_loss: 0.0300 - val_acc: 0.9914
Epoch 6/20
60000/60000 [==============================] - 10s 162us/step - loss: 0.0100 - acc: 0.9970 - val_loss: 0.0208 - val_acc: 0.9931
Epoch 7/20
60000/60000 [==============================] - 10s 163us/step - loss: 0.0074 - acc: 0.9976 - val_loss: 0.

In [7]:
 export_model(tf.train.Saver(), model, ["conv2d_7_input"], "dense_6/Softmax")

INFO:tensorflow:Restoring parameters from out/1608_mnist_convnet.chkp
INFO:tensorflow:Froze 10 variables.
INFO:tensorflow:Converted 10 variables to const ops.
graph saved!
